In [ ]:
import torch
import pandas as pd
from vllm import LLM, SamplingParams

torch.cuda.empty_cache()
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [2]:
model_name = "dmis-lab/llama-3-meerkat-8b-v1.0"


In [ ]:

# Initialize vLLM with the model
llm = LLM(model=model_name,
    dtype="bfloat16",
    gpu_memory_utilization=0.9,
    max_model_len=2048,
    trust_remote_code=True,
    tensor_parallel_size=1
)

tokenizer = llm.get_tokenizer()



INFO 02-16 19:18:18 __init__.py:190] Automatically detected platform cuda.
INFO 02-16 19:18:23 config.py:542] This model supports multiple tasks: {'score', 'classify', 'reward', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 02-16 19:18:23 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='dmis-lab/llama-3-meerkat-8b-v1.0', speculative_config=None, tokenizer='dmis-lab/llama-3-meerkat-8b-v1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 02-16 19:18:27 model_runner.py:1115] Loading model weights took 14.9595 GB
INFO 02-16 19:18:28 worker.py:267] Memory profiling takes 0.55 seconds
INFO 02-16 19:18:28 worker.py:267] the current vLLM instance can use total_gpu_memory (47.50GiB) x gpu_memory_utilization (0.90) = 42.75GiB
INFO 02-16 19:18:28 worker.py:267] model weights take 14.96GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 26.52GiB.
INFO 02-16 19:18:28 executor_base.py:110] # CUDA blocks: 13578, # CPU blocks: 2048
INFO 02-16 19:18:28 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 106.08x
INFO 02-16 19:18:30 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:11<00:00,  3.02it/s]

INFO 02-16 19:18:42 model_runner.py:1562] Graph capturing finished in 12 secs, took 0.26 GiB
INFO 02-16 19:18:42 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 14.77 seconds


In [ ]:
sampling_params = SamplingParams(temperature=0.5, top_p=0.95, max_tokens=500)


In [17]:
idf = pd.read_csv('medhalt_generic_new.csv')
input_df = idf[0:200].copy()
print(len(input_df))

200


In [18]:
messages = [
    {"role": "system", 
    "content": '''"The following is a multiple-choice question about medical knowledge. Solve this in"
    " a step-by-step fashion, starting by summarizing the available information. Output"
    " a single option from the given options as the final answer. You are strongly"
    " required to follow the specified output format; conclude your response with the"
    ' phrase "the answer is ([option_id]) [answer_string]".\n\n'
    '''},
]

In [19]:
def answer_question(question, options, answer, message_for_answering):

    qa = f"Question: {question}\n Options: {options}\n"
    message_for_answering.append({
    "role": "user",
    "content": qa,
    })
    #print(message_for_answering)
    message_for_answering = str(message_for_answering)
    
    try:
        response = llm.generate(message_for_answering, sampling_params)

        return response[0].outputs[0].text
    except Exception as e:
        print(f"Error: {e}")
        return None

def count_tokens(text: str) -> int:
    tokens = encoding.encode(text)
    return len(tokens)


In [ ]:

inputs, labels = [], []
results = []
for index, row in input_df.iterrows():
    question_id = row['id']
    question = row['question']
    option = row['options']
    correct_answer = row['correct_answer']
    correct_index = row['correct_index']
    content = tokenizer.apply_chat_template(
        [{"role": "system", "content": messages}, {"role": "user", "content": question + " " + option}],
        add_generation_prompt=True,
        tokenize=False,
    )
    inputs.append(content)
    labels.append(correct_index)
    results.append({
        "qid": question_id,
        "question": question,
        "options": row['options'],
        "answer": correct_answer,
    })

generated = llm.generate(
    inputs,
    SamplingParams(
        temperature=0.0,
        stop_token_ids=[tokenizer.vocab["<|eot_id|>"]],
        max_tokens=1024,
    ),
)

In [21]:
import re

def extract_answer(text: str, options: str = "ABCD") -> str:
    return (re.findall(rf"he answer is \(([{options}])\)", text) or [options[0]])[-1]

def extract_numeric_answer(text: str) -> str:
    match = re.findall(r"he answer is \((\d+)\)", text)
    return match[-1] if match else -1

In [ ]:
correctness = []

for g, l, r in zip(generated, labels, results):
    extracted = extract_numeric_answer(g.outputs[0].text)
    correctness.append( int(extracted) == int(l))
    r['correct_index'] = l
    r['reponse_index'] = extracted
    r['response'] = g.outputs[0].text
    print(l, extracted)
    

print(sum(correctness) / len(correctness))

In [ ]:
output_df = pd.DataFrame(results)

In [ ]:
mismatched_df = output_df[output_df["correct_index"].astype(str) != output_df["reponse_index"].astype(str)]
output_df.to_csv("path_to_save_incorrect_list.csv")

# calculate the performance from the results